In [ ]:
# VacationPy

In [ ]:
## Setting up Dependencies

In [1]:
#### Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from config import g_key

#configure gmaps
gmaps.configure(api_key=g_key)

In [ ]:
## Storing part one results into a DataFrame

In [2]:
#### Load CSV file generated from WeatherPy Folder
vacation_df = "../WeatherPy/output_data/cities.csv"
vacation_df = pd.read_csv(vacation_df)
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.65,115.33,54.00,54,0,10.56,AU,1603916711
1,1,Port Alfred,-33.59,26.89,66.22,86,86,2.71,ZA,1603916711
2,2,Mamit,23.93,92.48,68.05,99,21,1.30,IN,1603916711
3,3,Carnarvon,-24.87,113.63,68.00,94,0,13.87,AU,1603916711
4,4,Tiksi,71.69,128.87,0.70,91,99,18.32,RU,1603916711
...,...,...,...,...,...,...,...,...,...,...
536,536,Davenda,53.56,119.32,19.99,85,6,0.51,RU,1603916747
537,537,Salalah,17.02,54.09,82.40,74,40,2.24,OM,1603916747
538,538,Gijang,35.24,129.21,46.40,65,0,2.24,KR,1603916747
539,539,North Bend,43.41,-124.22,73.00,63,1,12.75,US,1603916747


In [ ]:
## Humidity Heat Map

In [19]:
#### Heatmap displaying Humidity for every city in vacation_df
locations = vacation_df[["Lat", "Lng"]].astype(float)
humidity = vacation_df["Humidity"].astype(float)
fig = gmaps.figure(center=(0.000, 0.000), zoom_level=1.9)


In [20]:
#### printing heat map
fig.add_layer(gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius = 3))
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
## Creating new DataFrame fitting Weather Criteria

In [25]:
#### Narrowing down the DataFrame to find ideal weather condition. 
#### A max temperature lower than 80 degrees but higher than 70.
#### Wind speed less than 10 mph.
#### Zero cloudiness.
#### Dropping any rows that don't contain all three conditions. 


new_vacation_df = vacation_df.loc[(vacation_df["Max Temp"]>70)&
                                  (vacation_df["Max Temp"]<80)&
                                  (vacation_df["Wind Speed"]<10)&
                                  (vacation_df["Cloudiness"]==0)].reset_index(drop=True)
new_vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,32,Castro,-24.79,-50.01,79.66,42,0,4.32,BR,1603916713
1,40,Umarkot,25.36,69.74,77.65,42,0,8.90,PK,1603916713
2,132,Sur,22.57,59.53,71.42,62,0,3.22,OM,1603916720
3,195,Puri,19.80,85.85,76.48,55,0,7.11,IN,1603916725
4,225,Guerrero Negro,27.98,-114.06,74.71,19,0,6.24,MX,1603916727
5,241,Cairns,-16.92,145.77,71.60,88,0,2.24,AU,1603916470
6,282,Al Bardīyah,31.76,25.09,70.90,68,0,9.46,LY,1603916732
7,290,Meghraj,23.50,73.50,74.77,36,0,5.48,IN,1603916732
8,306,Cap Malheureux,-19.98,57.61,72.00,74,0,5.99,MU,1603916733
9,338,Birao,10.28,22.79,78.82,34,0,4.21,CF,1603916735


In [ ]:
## Hotel Map

In [ ]:
### creating Dataframe with only needed fields

In [26]:
hotel_df = pd.DataFrame(new_vacation_df, columns=["City", "Country", "Lat", "Lng"])
hotel_df

,City,Country,Lat,Lng
0,Castro,BR,-24.79,-50.01
1,Umarkot,PK,25.36,69.74
2,Sur,OM,22.57,59.53
3,Puri,IN,19.80,85.85
4,Guerrero Negro,MX,27.98,-114.06
5,Cairns,AU,-16.92,145.77
6,Al Bardīyah,LY,31.76,25.09
7,Meghraj,IN,23.50,73.50
8,Cap Malheureux,MU,-19.98,57.61
9,Birao,CF,10.28,22.79


In [ ]:
### Finding First Hotel that is in 5000 meters radius from the cities

In [27]:
#### set up lists for hotel information
hotelName = []
hotelAddress = []

for index, row in hotel_df.iterrows():
    #current info
    currentCity = row["City"]
    currentCountry = row["Country"]
    
    #geocoordinates
    target_coordinates = f"{row['Lat']},{row['Lng']}"
    target_radius = 5000 
    target_type = "hotel"
    target_keyword = "hotel"

    #set parameters
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "keyword": target_keyword,
        "key": g_key
    }
    
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #run request using params dictionary
    response = requests.get(base_url,params=params).json()
    #simplify finding results
    results = response['results']
     
    try:
        hotelName.append(results[0]['name'])
        hotelAddress.append(results[0]['vicinity'])
        print(f"Processeing Record for a {target_keyword} in {currentCity}, {currentCountry}")
    except:
        hotelName.append(np.nan)
        hotelAddress.append(np.nan)
        print(f"No results/{target_keyword} found in {currentCity}, {currentCountry}. Skipping...")


Processeing Record for a hotel in Castro, BR
Processeing Record for a hotel in Umarkot, PK
Processeing Record for a hotel in Sur, OM
Processeing Record for a hotel in Puri, IN
Processeing Record for a hotel in Guerrero Negro, MX
Processeing Record for a hotel in Cairns, AU
Processeing Record for a hotel in Al Bardīyah, LY
No results/hotel found in Meghraj, IN. Skipping...
Processeing Record for a hotel in Cap Malheureux, MU
No results/hotel found in Birao, CF. Skipping...
Processeing Record for a hotel in Jaumave, MX
Processeing Record for a hotel in Tura, IN
Processeing Record for a hotel in Ures, MX
Processeing Record for a hotel in Koungou, YT
Processeing Record for a hotel in Dzaoudzi, YT
Processeing Record for a hotel in Noumea, NC


In [ ]:
### Additing results to a new DataFrame

In [30]:
#### add new information into hotel_df
hotel_df["Hotel Name"] = hotelName
hotel_df["Hotel Address"] = hotelAddress
hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel Address
0,Castro,BR,-24.79,-50.01,Hotel Buganville Express,"R. Mariana Marquês, 66 - Centro, Castro"
1,Umarkot,PK,25.36,69.74,Ghosia Guest House,"near eye hospital، Water Supply Rd, Umarkot"
2,Sur,OM,22.57,59.53,Sur Plaza Hotel,Sur
3,Puri,IN,19.80,85.85,"MAYFAIR Heritage, Puri","Plot No 122, 124, 125, Chakra Tirtha Rd, Puri"
4,Guerrero Negro,MX,27.98,-114.06,The Halfway Inn,"Paralelo 28, México 1, Guerrero Negro"
5,Cairns,AU,-16.92,145.77,"Shangri-La Hotel, The Marina, Cairns","Pier Point Rd, Cairns City"
6,Al Bardīyah,LY,31.76,25.09,Hotel Burdy,Libya
7,Meghraj,IN,23.50,73.50,NaN,NaN
8,Cap Malheureux,MU,-19.98,57.61,Lagoon Attitude Hotel,Royal Road Anse La Raie
9,Birao,CF,10.28,22.79,NaN,NaN


In [31]:
#### remove NaN rows
hotelFound_df = hotel_df.dropna()
hotelFound_df

,City,Country,Lat,Lng,Hotel Name,Hotel Address
0,Castro,BR,-24.79,-50.01,Hotel Buganville Express,"R. Mariana Marquês, 66 - Centro, Castro"
1,Umarkot,PK,25.36,69.74,Ghosia Guest House,"near eye hospital، Water Supply Rd, Umarkot"
2,Sur,OM,22.57,59.53,Sur Plaza Hotel,Sur
3,Puri,IN,19.80,85.85,"MAYFAIR Heritage, Puri","Plot No 122, 124, 125, Chakra Tirtha Rd, Puri"
4,Guerrero Negro,MX,27.98,-114.06,The Halfway Inn,"Paralelo 28, México 1, Guerrero Negro"
5,Cairns,AU,-16.92,145.77,"Shangri-La Hotel, The Marina, Cairns","Pier Point Rd, Cairns City"
6,Al Bardīyah,LY,31.76,25.09,Hotel Burdy,Libya
8,Cap Malheureux,MU,-19.98,57.61,Lagoon Attitude Hotel,Royal Road Anse La Raie
10,Jaumave,MX,23.42,-99.38,HOTEL CIENEGA,Jaumave
11,Tura,IN,25.52,90.22,Hotel Polo Orchid,"Araimile, Tura"


In [ ]:
### Info Box

In [32]:

#### Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotelFound_info = [info_box_template.format(**row) for index, row in hotelFound_df.iterrows()]
locations = hotelFound_df[["Lat", "Lng"]]

In [ ]:
### Plot Hotel Info Box on top of Humidity Heat Map

In [33]:
#### Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotelFound_info)
fig.add_layer(markers)

#### Display figure
fig

Figure(layout=FigureLayout(height='420px'))